<a href="https://colab.research.google.com/github/pranavgovi/IBM_ASSESSMENT_PNT2022TMID53243/blob/main/ibm_healthcare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:

import numpy as np
import pandas as pd 
import os
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,StratifiedKFold,RepeatedStratifiedKFold
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')
# import eli5
# from eli5.sklearn import PermutationImportance
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [3]:
train_df      = pd.read_csv('/content/drive/MyDrive/Pitt/ibm/train.csv')
test_df       =  pd.read_csv('/content/drive/MyDrive/Pitt/ibm/test.csv')
submission_df = pd.read_csv('/content/drive/MyDrive/Pitt/ibm/sample_submission.csv')


In [4]:
#Creating Addition Features
combine_set=pd.concat([train_df,test_df],axis=0)
combine_set['City_Code_Patient'].fillna(-99,inplace=True)
combine_set['Bed Grade'].fillna(5,inplace=True)
combine_set['Unique_Hospital_per_patient']=combine_set.groupby(['patientid'])['Hospital_code'].transform('nunique')
combine_set['Unique_patient_per_hospital']=combine_set.groupby(['Hospital_code'])['patientid'].transform('nunique')
combine_set['Unique_patient_per_Department']=combine_set.groupby(['Department'])['patientid'].transform('nunique')
combine_set['Total_deposit_paid_by_patient_in_each_hospital']=combine_set.groupby(['Hospital_code','patientid'])['Admission_Deposit'].transform('sum')
combine_set['Min_Severity_of_Illness'] = combine_set.groupby('patientid')['Severity of Illness'].transform('min')

In [5]:
# In[4]:

le = LabelEncoder()
#Encoding categorical variables by frequency encoding and label encoding
for col in combine_set.select_dtypes(include='object').columns:
    if col not in ['Age','Stay']:
        fe=combine_set.groupby([col]).size()/len(combine_set)
        combine_set[col]=combine_set[col].apply(lambda x: fe[x])   
        # combine_set[col]  = pd.get_dummies(combine_set[col].astype(str))         
    elif col!='Stay':
        combine_set[col]=le.fit_transform(combine_set[col].astype(str))
    else:
        pass

In [6]:
# In[5]:

#Splitting train and test

X=combine_set[combine_set['Stay'].isnull()==False].drop(['case_id','Stay'],axis=1)
y=le.fit_transform(combine_set[combine_set['Stay'].isnull()==False]['Stay'])
y=pd.DataFrame(y,columns=['Stay'])
X_main_test=combine_set[combine_set['Stay'].isnull()==True].drop(['case_id','Stay'],axis=1)

In [7]:
# In[6]:

# kf=KFold(n_splits=10,shuffle=True,random_state=294)
kf=KFold(n_splits=5,shuffle=True)

preds_1   = {}
y_pred_1  = []
acc_score = []

for i,(train_idx,val_idx) in enumerate(kf.split(X)):    
    
    X_train, y_train = X.iloc[train_idx,:], y.iloc[train_idx]
    
    X_val, y_val = X.iloc[val_idx, :], y.iloc[val_idx]
   
    print('\nFold: {}\n'.format(i+1))

    lg=LGBMClassifier( 
                      gpu_platform_id= 0,
                      max_bin=63,#Theoretically best speeds using LGBM
                      gpu_device_id= 0,
                      boosting_type='gbdt',
                      learning_rate=0.04,
                      # max_depth=15,
                      # num_leaves = 150,
                      objective='multi_class',
                      num_class=11,                      
                      n_estimators=50000,
                      metric='multi_error',
                      colsample_bytree=0.8,
                      min_child_samples=228,
                      reg_alpha=1,
                      reg_lambda=1,
                      # random_state=294,
                      n_jobs=-1,
                     
                      ) 
    
    # lg.fit(X_train,y_train)
    lg.fit(X_train, y_train
#                         ,categorical_feature = categorical_features
                        ,eval_metric='multi_error'
                        ,eval_set=[(X_train, y_train),(X_val, y_val)]
                        ,early_stopping_rounds=100
                        ,verbose=50
                       )
    
    print(accuracy_score(y_val,lg.predict(X_val))*100)
    
    acc = accuracy_score(y_val,lg.predict(X_val))*100
    acc_score.append(acc)
    print("Score : ",acc)    
    y_pred_1.append(lg.predict_proba(X_main_test))
    
    # preds_1[i+1]=lg.predict_proba(X_main_test)
    # y_pred_1.append(lg.predict_proba(X_main_test))

y_pred_final_1          = np.mean(np.array(y_pred_1),axis=0)
    
print('mean accuracy score: {}'.format((sum(acc_score)/10)))

preds_1=np.argmax(y_pred_final_1,axis=1)

print(preds_1.shape)
submission_df['Stay']=le.inverse_transform(preds_1.astype(int))
# submission_df[0] = y_pred_final_1[:,0]
# submission_df[1] =y_pred_final_1[:,1]

# Download Submission File :
display("submission_df",submission_df)
sub_file_name_1 = "BEST_11_CV=42.96_LB=WAIT_LGBM-1.csv"

submission_df.to_csv(sub_file_name_1,index=False)
submission_df.head(5)


Fold: 1

Training until validation scores don't improve for 100 rounds.
[50]	training's multi_error: 0.58546	valid_1's multi_error: 0.589436
[100]	training's multi_error: 0.575713	valid_1's multi_error: 0.580753
[150]	training's multi_error: 0.570599	valid_1's multi_error: 0.57788
[200]	training's multi_error: 0.566818	valid_1's multi_error: 0.575917
[250]	training's multi_error: 0.563584	valid_1's multi_error: 0.574912
[300]	training's multi_error: 0.560938	valid_1's multi_error: 0.573891
[350]	training's multi_error: 0.558807	valid_1's multi_error: 0.573452
[400]	training's multi_error: 0.556467	valid_1's multi_error: 0.572839
[450]	training's multi_error: 0.554273	valid_1's multi_error: 0.572274
[500]	training's multi_error: 0.5518	valid_1's multi_error: 0.572573
[550]	training's multi_error: 0.549472	valid_1's multi_error: 0.572101
[600]	training's multi_error: 0.547274	valid_1's multi_error: 0.572117
[650]	training's multi_error: 0.54515	valid_1's multi_error: 0.571819
[700]	trai

'submission_df'

,case_id,Stay
0,318439,0-10
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,51-60
...,...,...
137052,455491,21-30
137053,455492,0-10
137054,455493,21-30
137055,455494,11-20


,case_id,Stay
0,318439,0-10
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,51-60


In [ ]:
pip install lightgbm --install-option -DUSE_GPU=1

/usr/local/lib/python3.8/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 35 kB/s 


In [10]:
from catboost                         import CatBoostClassifier
import timeit

catboost = CatBoostClassifier(eval_metric='Accuracy', max_depth=4, task_type="GPU", devices="0:1", n_estimators=1000, verbose=500)
catboost.fit( X, y, verbose=10 )   
y_pred_3  = []
y_pred_3.append(catboost.predict_proba(X_main_test))
y_pred_final_3          = np.mean(np.array(y_pred_3),axis=0)
# gpu_time = timeit.timeit('train_on_gpu()', setup="from __main__ import train_on_gpu", number=1)
# print('Time to fit and predict model on GPU: {} sec'.format(int(gpu_time)))

preds_3=np.argmax(y_pred_final_3,axis=1)

submission_df['Stay']=le.inverse_transform(preds_3.astype(int))
# submission_df[0] = y_pred_final_1[:,0]
# submission_df[1] =y_pred_final_1[:,1]

# Download Submission File :
display("submission_df",submission_df)
sub_file_name_3 = "BEST_11_CV=42.96_LB=WAIT_LGBM-1.csv"

submission_df.to_csv(sub_file_name_3,index=False)
submission_df.head(5)

Learning rate set to 0.21033
0:	learn: 0.3383484	total: 16.8ms	remaining: 16.8s
10:	learn: 0.3866781	total: 326ms	remaining: 29.3s
20:	learn: 0.3996225	total: 517ms	remaining: 24.1s
30:	learn: 0.4075330	total: 673ms	remaining: 21s
40:	learn: 0.4111004	total: 791ms	remaining: 18.5s
50:	learn: 0.4140523	total: 905ms	remaining: 16.8s
60:	learn: 0.4161721	total: 1.01s	remaining: 15.5s
70:	learn: 0.4185713	total: 1.11s	remaining: 14.6s
80:	learn: 0.4202733	total: 1.22s	remaining: 13.8s
90:	learn: 0.4214478	total: 1.32s	remaining: 13.2s
100:	learn: 0.4224119	total: 1.43s	remaining: 12.7s
110:	learn: 0.4231844	total: 1.53s	remaining: 12.3s
120:	learn: 0.4241014	total: 1.65s	remaining: 12s
130:	learn: 0.4248676	total: 1.75s	remaining: 11.6s
140:	learn: 0.4256150	total: 1.86s	remaining: 11.4s
150:	learn: 0.4261991	total: 1.98s	remaining: 11.1s
160:	learn: 0.4267236	total: 2.08s	remaining: 10.9s
170:	learn: 0.4271601	total: 2.23s	remaining: 10.8s
180:	learn: 0.4275087	total: 2.33s	remaining: 10.

'submission_df'

,case_id,Stay
0,318439,0-10
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,51-60
...,...,...
137052,455491,21-30
137053,455492,0-10
137054,455493,11-20
137055,455494,11-20


,case_id,Stay
0,318439,0-10
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,51-60


In [11]:
# Ensemble of LGBM + CatBoost :

preds = (y_pred_final_1*0.2 + y_pred_final_3*0.8) /2
preds=np.argmax(preds,axis=1)
print(preds)

# In[9]:
# Download Submission File :
submission_df['Stay']=le.inverse_transform(preds.astype(int))
display("submission_df",submission_df)
sub_file_name = "ENSEMBLE_1_CV=42.22_42.17_LB=WAIT_LGB-1_0.2_LBG-2_0.8.csv"

submission_df.to_csv(sub_file_name,index=False)
submission_df.head(5)

[0 5 2 ... 2 1 5]


'submission_df'

,case_id,Stay
0,318439,0-10
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,51-60
...,...,...
137052,455491,21-30
137053,455492,0-10
137054,455493,21-30
137055,455494,11-20


,case_id,Stay
0,318439,0-10
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,51-60
